In [4]:
from scipy.io import arff
import pandas as pd
df_train = pd.read_csv('base_treinamento.csv')
df_test = pd.read_csv('base_teste.csv')

#pegando o valor absoluto pois todos os valores dos AP são negativos e as coordenadas sao sempre positivas
df_train = df_train.abs()
df_test = df_test.abs()

X_train = df_train[['AP1','AP2','AP3','AP4','AP5']]
X_test = df_test[['AP1','AP2','AP3','AP4','AP5']]
y_train = df_train[['Coordenada X','Coordenada Y']]
y_test = df_test[['Coordenada X','Coordenada Y']]

X_train =X_train/100
X_test = X_test/100

X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

df_train.describe()


,AP1,AP2,AP3,AP4,AP5,Coordenada X,Coordenada Y
count,262.00000,262.000000,262.000000,262.000000,262.000000,262.000000,262.000000
mean,67.90458,62.129771,58.122137,71.454198,65.183206,12.026985,12.612634
std,10.29165,8.530672,8.323924,9.232823,7.128596,5.079505,5.139619
min,38.00000,35.000000,33.000000,38.000000,51.000000,1.390000,1.570000
25%,61.00000,57.000000,53.000000,66.250000,60.000000,8.442500,9.412500
50%,69.00000,63.000000,58.000000,74.000000,65.000000,12.400000,13.630000
75%,75.75000,68.000000,65.000000,78.000000,69.000000,16.357500,16.710000
max,90.00000,81.000000,75.000000,86.000000,84.000000,20.450000,19.630000


In [2]:
bp = df_train.boxplot(figsize=(10,10))
bp

In [3]:
y_train.shape

(262, 2)

In [185]:
from sklearn.neural_network import MLPRegressor
clf = MLPRegressor(random_state=1,max_iter=10000,hidden_layer_sizes=(1,))
clf.fit(X_train,y_train)



MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=10000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [186]:
import numpy as np
y_pred = clf.predict(X_test)#testando
distances = []
for i in range(y_pred.shape[0]):
    predx = y_pred[i][0]
    predy = y_pred[i][1]
    tesx = y_test[i][0]
    tesy = y_test[i][1]
    distance = (float((predx-tesx)**2) +  float((predy-tesy)**2))**0.5
    distances.append(distance)
print(np.mean(distances))

6.575409003186499


In [146]:
parameter_space = {
    'max_iter' : [10**3,10**4],
    'hidden_layer_sizes': [(5,5,5), (5,10,5), (5,),(2,),(1,),(10)],
    'activation': ['tanh', 'relu','tanh','identity'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
}

In [147]:
from sklearn.model_selection import GridSearchCV
gridclf = GridSearchCV(clf, parameter_space, n_jobs=-4, cv=5)
gridclf.fit(X_train, y_train)

C:\Users\Flavio\Anaconda3\envs\test\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=10000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=3, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-4,
       param_grid={'max_iter': [1000, 10000], 'hidden_layer_sizes': [(5, 5, 5), (5, 10, 5), (5,), (2,), (1,), 10], 'activation': ['tanh', 'relu', 'tanh', 'identity'], 'solver': ['sgd', 'adam'], 'alpha': [0.0001, 0.05]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [152]:
# Best paramete set
print('Best parameters found:\n', gridclf.best_params_)

Best parameters found:
 {'activation': 'identity', 'alpha': 0.0001, 'hidden_layer_sizes': (5, 5, 5), 'max_iter': 1000, 'solver': 'sgd'}


In [13]:
test =X_train.reshape(131,5,2)
test

array([[[0.45, 0.64],
        [0.59, 0.73],
        [0.51, 0.42],
        [0.69, 0.62],
        [0.78, 0.59]],

       [[0.56, 0.71],
        [0.61, 0.85],
        [0.61, 0.54],
        [0.72, 0.63],
        [0.77, 0.6 ]],

       [[0.58, 0.69],
        [0.64, 0.81],
        [0.63, 0.63],
        [0.7 , 0.62],
        [0.76, 0.65]],

       ...,

       [[0.8 , 0.76],
        [0.6 , 0.59],
        [0.79, 0.84],
        [0.79, 0.63],
        [0.6 , 0.68]],

       [[0.8 , 0.72],
        [0.63, 0.51],
        [0.72, 0.75],
        [0.64, 0.59],
        [0.48, 0.69]],

       [[0.75, 0.75],
        [0.56, 0.56],
        [0.77, 0.74],
        [0.76, 0.58],
        [0.52, 0.66]]])

In [2]:
#Load Packages
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
#Generate 2 sets of X variables
#LSTMs have unique 3-dimensional input requirements 
seq_length=5
X =[[i+j for j in range(seq_length)] for i in range(100)]
X_simple =[[i for i in range(4,104)]]
X =np.array(X)
X_simple=np.array(X_simple)

Using TensorFlow backend.
